# Neural Networks

In [1]:
import warnings
# Suppress all warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

In [3]:
# Load the data
df = pd.read_csv('BankRecords.csv')
# df.head()


## Data Preparation

In [5]:
# Renaming some of the columns to remove especial characters
df = df.rename(columns={
    'Experience(Years)': 'Experience_years',
    'Income(Thousands\'s)': 'Income_thousands',
    'Mortgage(Thousands\'s)' : 'Mortgage_thousands'
})

In [6]:
# Remove null values
df.dropna(inplace=True)

## Checking df's characteristics

In [7]:
# Checking dimensions
df.shape

(5000, 14)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   Age                 5000 non-null   int64  
 2   Experience_years    5000 non-null   int64  
 3   Income_thousands    5000 non-null   int64  
 4   Sort Code           5000 non-null   int64  
 5   Family              5000 non-null   int64  
 6   Credit Score        5000 non-null   float64
 7   Education           5000 non-null   object 
 8   Mortgage_thousands  5000 non-null   int64  
 9   Personal Loan       5000 non-null   object 
 10  Securities Account  5000 non-null   object 
 11  CD Account          5000 non-null   object 
 12  Online Banking      5000 non-null   object 
 13  CreditCard          5000 non-null   object 
dtypes: float64(1), int64(7), object(6)
memory usage: 547.0+ KB


In [9]:
df.describe()

,ID,Age,Experience_years,Income_thousands,Sort Code,Family,Credit Score,Mortgage_thousands
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,2500.500000,45.338400,20.104600,73.774200,93152.503000,2.396400,1.937913,56.498800
std,1443.520003,11.463166,11.467954,46.033729,2121.852197,1.147663,1.747666,101.713802
min,1.000000,23.000000,-3.000000,8.000000,9307.000000,1.000000,0.000000,0.000000
25%,1250.750000,35.000000,10.000000,39.000000,91911.000000,1.000000,0.700000,0.000000
50%,2500.500000,45.000000,20.000000,64.000000,93437.000000,2.000000,1.500000,0.000000
75%,3750.250000,55.000000,30.000000,98.000000,94608.000000,3.000000,2.500000,101.000000
max,5000.000000,67.000000,43.000000,224.000000,96651.000000,4.000000,10.000000,635.000000


## Encoding categorical variables

In [10]:
categorical_columns = ['Education', 'Personal Loan', 'Securities Account', 'CD Account', 'Online Banking', 'CreditCard']
df_encoded = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

In [11]:
from sklearn.model_selection import train_test_split

# Define the feature matrix X and the target vector y
X = df_encoded.drop('Income_thousands', axis=1)
y = df_encoded['Income_thousands']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)


In [12]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the testing data
X_test_scaled = scaler.transform(X_test)


## NN Model

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the neural network model
model_nn = Sequential()
model_nn.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))
model_nn.add(Dense(32, activation='relu'))
model_nn.add(Dense(1))

# Compile the model
model_nn.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

# Train the model
model_nn.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test), epochs=100, batch_size=32)
# Evaluate the neural network model
mae_nn = model_nn.evaluate(X_test_scaled, y_test)[1]


Epoch 1/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 6983.4082 - mean_absolute_error: 70.4851 - val_loss: 3721.8237 - val_mean_absolute_error: 47.9908
Epoch 2/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2411.6873 - mean_absolute_error: 37.3730 - val_loss: 951.3931 - val_mean_absolute_error: 24.7724
Epoch 3/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1023.3388 - mean_absolute_error: 24.8313 - val_loss: 909.7786 - val_mean_absolute_error: 24.2192
Epoch 4/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 951.1308 - mean_absolute_error: 24.3601 - val_loss: 893.5128 - val_mean_absolute_error: 23.9577
Epoch 5/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 934.5800 - mean_absolute_error: 24.2027 - val_loss: 879.4808 - val_mean_absolute_error: 23.7065
Epoch 6/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 894.6521 - mean_absolute_error: 23.6036 - val_loss: 872.5609 - val_mean_absolute_error: 23.7619
Epoch 7/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step -

## Random Forest

In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Initialize and train the Random Forest Regressor
model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)

# Predict and evaluate the model
y_pred_rf = model_rf.predict(X_test)
mae_rf = mean_absolute_error(y_test, y_pred_rf)


### Results

In [15]:

print(f'Mean Absolute Error (Neural Network): {mae_nn}')
print(f'Mean Absolute Error (Random Forest): {mae_rf}')


Mean Absolute Error (Neural Network): 21.568492889404297
Mean Absolute Error (Random Forest): 15.809949999999999


In [16]:
# !pip install tensorflow==2.14.0
# !pip install keras==2.14.0

# !pip install --upgrade tensorflow scikit-learn scikeras
# !pip install tensorflow scikit-learn scikeras --ignore-installed


# Models Optimizations

## Neural Networks

In [17]:
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [18]:
# Define a function to create the neural network model
def create_model(neurons=32, activation='relu', optimizer='adam'):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X_train_scaled.shape[1], activation=activation))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_absolute_error'])
    return model

# Wrap the model using KerasRegressor
model_nn2 = KerasRegressor(model=create_model, verbose=0)

# Define the parameter grid
param_grid = {
    'batch_size': [16, 32],
    'epochs': [50, 100],
    'model__optimizer': ['adam'],
    'model__activation': ['relu'],
    'model__neurons': [32, 64]
}


In [19]:
# Perform Grid Search
grid_search = GridSearchCV(estimator=model_nn2, param_grid=param_grid, scoring='neg_mean_absolute_error', n_jobs=-1, cv=3)
grid_result = grid_search.fit(X_train_scaled, y_train)

# Display the best parameters
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")


Best: -21.80175956827965 using {'batch_size': 32, 'epochs': 50, 'model__activation': 'relu', 'model__neurons': 64, 'model__optimizer': 'adam'}


In [20]:
# Perform Randomized Search
random_search = RandomizedSearchCV(estimator=model_nn2, param_distributions=param_grid, n_iter=20, scoring='neg_mean_absolute_error', n_jobs=-1, cv=3, random_state=42)
random_result = random_search.fit(X_train_scaled, y_train)

# Display the best parameters
print(f"Best: {random_result.best_score_} using {random_result.best_params_}")


Best: -21.856003531898182 using {'model__optimizer': 'adam', 'model__neurons': 32, 'model__activation': 'relu', 'epochs': 100, 'batch_size': 32}


## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Initialize the Random Forest Regressor
rf = RandomForestRegressor(random_state=42)

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform Grid Search
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='neg_mean_absolute_error', n_jobs=-1, cv=3)
grid_result_rf = grid_search_rf.fit(X_train, y_train)

# Display the best parameters
print(f"Best: {grid_result_rf.best_score_} using {grid_result_rf.best_params_}")


In [ ]:
# Perform Randomized Search
random_search_rf = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, n_iter=20, scoring='neg_mean_absolute_error', n_jobs=-1, cv=3, random_state=42)
random_result_rf = random_search_rf.fit(X_train, y_train)

# Display the best parameters
print(f"Best: {random_result_rf.best_score_} using {random_result_rf.best_params_}")


## New customers predictions

In [ ]:
import random

In [ ]:
# Function to generate random customer data
def generate_customer():
    return {
        'Age': random.randint(18, 80),
        'Experience(Years)': random.randint(1, 50),
        'Family': random.randint(1, 5),
        'Credit Score': round(random.uniform(0.5, 4.0), 2),
        'Education': random.choice(['Diploma', 'Degree', 'Masters']),
        'Mortgage': random.randint(0, 500),
        'Personal Loan': random.choice(['Yes', 'No']),
        'Securities Account': random.choice(['Yes', 'No']),
        'CD Account': random.choice(['Yes', 'No']),
        'Online Banking': random.choice(['Yes', 'No']),
        'CreditCard': random.choice(['Yes', 'No'])
    }

In [ ]:
# Generate 20 new customers
new_customers = [generate_customer() for _ in range(20)]

# Convert new customers to DataFrame
new_customers_df = pd.DataFrame(new_customers)

# Encode categorical variables
new_customers_encoded = pd.get_dummies(new_customers_df, columns=categorical_columns, drop_first=True)

# Ensure all columns are present
new_customers_encoded = new_customers_encoded.reindex(columns=X.columns, fill_value=0)

# Scale features
new_customers_scaled = scaler.transform(new_customers_encoded)

# Make predictions
predicted_incomes = model_nn.predict(new_customers_scaled)

# Add predicted incomes to the new customers DataFrame
new_customers_df['Predicted Income'] = predicted_incomes

# Display the new customers with their predicted incomes
new_customers_df


# Sentiment Analysis - Game of thrones Tweets

In [ ]:
import re
from transformers import pipeline
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load a larger chunk of the data to enable random sampling
df_full = pd.read_csv('gotTwitter.csv', usecols=['text', 'created_at', 'screen_name'])


In [ ]:
# Select a random subset of 2000 rows
df_got = df_full.sample(n=1000, random_state=35).reset_index(drop=True)

# Display the first few rows of the random subset
df_got.head()


In [ ]:
# Function to clean the tweet text
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'@\w+', '', text)  # Remove mentions
    text = re.sub(r'#\w+', '', text)  # Remove hashtags
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.lower()  # Convert to lowercase
    return text

# Apply the cleaning function to the tweet text
df_got['cleaned_text'] = df_got['text'].apply(clean_text)

In [ ]:
# Load the sentiment analysis pipeline with a specified model
sentiment_pipeline = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')


In [ ]:
# Function to map model output to positive, neutral, and negative sentiments
def map_sentiment(label):
    score = int(label.split()[0])  # Extract the score from the label (e.g., '3 stars')
    if score == 1 or score == 2:
        return 'negative'
    elif score == 3:
        return 'neutral'
    else:
        return 'positive'

# Apply the sentiment analysis pipeline to the cleaned text with progress bar
tqdm.pandas()
df_got['sentiment'] = df_got['cleaned_text'].progress_apply(lambda x: map_sentiment(sentiment_pipeline(x)[0]['label']))


In [ ]:
# Display the first few sentiment results
df_got[['cleaned_text', 'sentiment']].head()

## Barchart of amount ot tweets per sentiment

In [ ]:
# Count the number of each sentiment
sentiment_counts = df_got['sentiment'].value_counts()

# Plot the sentiment distribution
plt.figure(figsize=(8, 6))
sns.barplot(x=sentiment_counts.index, y=sentiment_counts.values, palette=['#ff9999','#66b3ff','#99ff99'])
plt.title('Sentiment Distribution of Game of Thrones Tweets')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.show()


## Piechart of sentiment distribution

In [ ]:
# Count the number of each sentiment
sentiment_counts = df_got['sentiment'].value_counts()

# Plot the pie chart
plt.figure(figsize=(8, 8))
plt.pie(sentiment_counts, labels=sentiment_counts.index, autopct='%1.1f%%', startangle=140, colors=['#ff9999','#66b3ff','#99ff99'])
plt.title('Sentiment Distribution of Game of Thrones Tweets')
# plt.axis('equal')  # Equal aspect ratio ensures the pie is drawn as a circle.
plt.show()


## Wordcloud

In [ ]:
# !pip install wordcloud

In [ ]:
from wordcloud import WordCloud

# Function to generate word cloud
def generate_wordcloud(text, title):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(title, fontsize=20)
    plt.axis('off')
    plt.show()

# Generate word clouds for different sentiments
positive_text = ' '.join(df_got[df_got['sentiment'] == 'positive']['cleaned_text'])
neutral_text = ' '.join(df_got[df_got['sentiment'] == 'neutral']['cleaned_text'])
negative_text = ' '.join(df_got[df_got['sentiment'] == 'negative']['cleaned_text'])

generate_wordcloud(positive_text, 'Positive Sentiment Words')
generate_wordcloud(neutral_text, 'Neutral Sentiment Words')
generate_wordcloud(negative_text, 'Negative Sentiment Words')


## Sentiment over time

In [ ]:
# Convert 'created_at' to datetime
df_got['created_at'] = pd.to_datetime(df_got['created_at'])

# Resample to get weekly sentiment counts
weekly_sentiment = df_got.set_index('created_at').resample('W')['sentiment'].value_counts().unstack().fillna(0)

# Plot the sentiment over time
weekly_sentiment.plot(kind='line', marker='o', figsize=(8, 4),color=['#ff9999','#66b3ff','#99ff99'])
plt.title('Sentiment Over Time')
plt.xlabel('Week')
plt.ylabel('Count')
plt.legend(title='Sentiment',bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()
